# Modelo Tesseract

En este notebook se presenta el estudio y resultados del modelo Tesseract de OCR para la region tipo texto.

Una de las mayores ventajas de Tesseract es su facilidad de uso.

In [1]:
import pytesseract
import cv2 as cv
from matplotlib import pyplot as plt
import torch
from jiwer import wer, cer
import os
import json
import pandas as pd

Cargamos el ground truth generado manualmente:

In [2]:
# Ruta del JSON de ground truth
gt_path = "D:\\Users\\Mauro\\TesisIA\\data\\annotations\\ocr_text_labels.json"

# Cargar el archivo como diccionario
with open(gt_path, "r", encoding="utf-8") as f:
    gt_dict = json.load(f)


Definimos las funciones de trabajo para obtener las métricas de comparación:

In [3]:
def limpiar_texto_ocr(texto):
    """
    Une palabras cortadas por guión al final de línea y reemplaza saltos de línea.
    """
    texto = texto.replace("-\n", "")         # Une palabras cortadas por guión
    texto = texto.replace("\n", " ")         # Resto de saltos de línea
    texto = " ".join(texto.split())          # Elimina espacios múltiples
    return texto.strip()

def contar_palabras(texto):
    return len(texto.split())

def obtener_vocabularios(gt_texto, ocr_texto):
    """
    Devuelve sets de vocabulario y diferencias, y calcula el Word Correct Count (wcc).
    
    wcc = len(solo_en_gt) / len(vocabulario_gt)
    """
    gt_vocab = set(gt_texto.split())
    ocr_vocab = set(ocr_texto.split())
    solo_en_gt = gt_vocab - ocr_vocab
    solo_en_ocr = ocr_vocab - gt_vocab

    wcc = len(solo_en_gt) / len(gt_vocab) if len(gt_vocab) > 0 else 0.0

    return {
        "solo_en_gt": list(solo_en_gt),
        "solo_en_ocr": list(solo_en_ocr),
        "wcc": wcc
    }

def process_ocr_entry_con_wordcount(image_dir, filename, ground_truth, config='--oem 3 --psm 6'):
    """
    Procesa una imagen OCR, corre Tesseract y compara con ground truth, incluyendo:
    - limpieza de texto
    - conteo de palabras
    - métricas WER y CER
    - vocabulario OCR vs GT
    """
    file_path = os.path.join(image_dir, filename)
    ocr_type = config 
    if not os.path.exists(file_path):
        print(f"Archivo no encontrado: {file_path}")
        return None

    img = cv.imread(file_path)
    if img is None:
        print(f"Error al leer imagen: {file_path}")
        return None

    # OCR
    extracted_text = pytesseract.image_to_string(img, config=config)

    # Limpieza
    extracted_text_clean = limpiar_texto_ocr(extracted_text)
    ground_truth_clean = limpiar_texto_ocr(ground_truth)

    # Métricas
    ocr_word_count = contar_palabras(extracted_text_clean)
    gt_word_count = contar_palabras(ground_truth_clean)
    cer_score = cer(ground_truth_clean, extracted_text_clean)
    wer_score = wer(ground_truth_clean, extracted_text_clean)

    # Vocabulario
    vocab_data = obtener_vocabularios(ground_truth_clean, extracted_text_clean)

    # Resultado
    return {
        "filename": filename,
        "ocr_text": extracted_text_clean,
        "ground_truth": ground_truth_clean,
        "wer": wer_score,
        "cer": cer_score,
        "ocr_word_count": ocr_word_count,
        "ground_truth_word_count": gt_word_count,
        "ocr_type": ocr_type,
        **vocab_data
    }


In [6]:
image_dir = "..\\data\\regions\\text\\"
results = []
ocr_config = ['--oem 3 --psm 6', '--oem 3 --psm 3', '--oem 3 --psm 7', '--oem 3 --psm 11']
for config in ocr_config:
    for filename, ground_truth in gt_dict.items():
        result = process_ocr_entry_con_wordcount(image_dir, filename, ground_truth, config= config)
        if result:
            results.append(result)

In [7]:
results

[{'filename': 'PMC1431532_00004_2861961.jpg',
  'ocr_text': 'cigarettes (CIG) are sufficient to control confounding and that there is no misclassification or other source of bias.',
  'ground_truth': 'cigarettes (CIG) are sufficient to control confounding and that there is no misclassification or other source of bias.',
  'wer': 0.0,
  'cer': 0.0,
  'ocr_word_count': 18,
  'ground_truth_word_count': 18,
  'ocr_type': '--oem 3 --psm 6',
  'solo_en_gt': [],
  'solo_en_ocr': [],
  'wcc': 0.0},
 {'filename': 'PMC1431532_00004_2861962.jpg',
  'ocr_text': 'longer provides a summary estimate of the odds ratio for the OC-MLI association.',
  'ground_truth': 'longer provides a summary estimate of the odds ratio for the OC-MI association.',
  'wer': 0.07692307692307693,
  'cer': 0.012658227848101266,
  'ocr_word_count': 13,
  'ground_truth_word_count': 13,
  'ocr_type': '--oem 3 --psm 6',
  'solo_en_gt': ['OC-MI'],
  'solo_en_ocr': ['OC-MLI'],
  'wcc': 0.08333333333333333},
 {'filename': 'PMC143

Lo anterior son los resultados con el modelo estándar de Tesseract. A continuación se trabaja con un poco del estudio de resultados, ¿que casos dió mejor? ¿que casos dió peor? que conclusiones podemos sacar.


## Análisis de resultados:

Primero vamos a generar un dataframe con los resultados obtenidos:

In [8]:
# Crear el DataFrame con las columnas deseadas
df_results = pd.DataFrame(results)[[
    'filename',
    'ground_truth_word_count',
    'ocr_word_count',
    'wer',
    'cer',
    'wcc',
    'ocr_type'
]]

In [11]:
df_avg = (
    df_results
    .drop(columns=["filename"])        # eliminar filename
    .groupby("ocr_type", as_index=False)
    .mean(numeric_only=True)           # promedios solo de columnas numéricas
)

df_avg


,ocr_type,ground_truth_word_count,ocr_word_count,wer,cer,wcc
0,--oem 3 --psm 11,69.531773,72.153846,0.143842,0.047990,0.094261
1,--oem 3 --psm 3,69.531773,70.040134,0.096560,0.035649,0.079144
2,--oem 3 --psm 6,69.531773,70.759197,0.104654,0.040297,0.077808
3,--oem 3 --psm 7,69.531773,2.622074,0.874084,0.848948,0.871965


In [9]:
# Guardar en CSV
df_results.to_csv("df_results.csv", index=False, encoding="utf-8")
